In [1]:
//  Microsoft 
// ❓Hire the candidates who fall under budget of 70000 according to below criteria:
// 📌 First hire Senior within budget
// 📌 Then hire Junior withing remaining budget.


val employees_df = Seq(
    (1,"Junior", 10000),
    (2,"Junior", 15000),
    (3,"Junior", 40000),
    (4,"Senior", 16000),
    (5,"Senior", 20000),
    (6,"Senior",50000)
).toDF("emp_id","experience","salary")

employees_df.show(false)


Intitializing Scala interpreter ...

Spark Web UI available at http://192.168.0.102:4040
SparkContext available as 'sc' (version = 3.4.1, master = local[*], app id = local-1706105504382)
SparkSession available as 'spark'


+------+----------+------+
|emp_id|experience|salary|
+------+----------+------+
|1     |Junior    |10000 |
|2     |Junior    |15000 |
|3     |Junior    |40000 |
|4     |Senior    |16000 |
|5     |Senior    |20000 |
|6     |Senior    |50000 |
+------+----------+------+



employees_df: org.apache.spark.sql.DataFrame = [emp_id: int, experience: string ... 1 more field]


In [8]:
employees_df.createOrReplaceTempView("employees")

spark.sql(""" 
    with cte as(
        select
            *,
            sum(salary) over(partition by experience order by emp_id) salary_by_experience
        from employees
     ),
     cte2 as(
         select
             *,
             sum(salary) over(order by case when experience='Senior' then 1 else 2 end, emp_id) total_budget
         from cte
         where salary_by_experience<=70000
    )
    select 
        emp_id,
        experience,
        salary
    from cte2
    where total_budget<=70000
    order by emp_id
""").show(false)


+------+----------+------+
|emp_id|experience|salary|
+------+----------+------+
|1     |Junior    |10000 |
|2     |Junior    |15000 |
|4     |Senior    |16000 |
|5     |Senior    |20000 |
+------+----------+------+



In [ ]:
// cte
// +------+----------+------+--------------------+
// |emp_id|experience|salary|salary_by_experience|
// +------+----------+------+--------------------+
// |1     |Junior    |10000 |10000               |
// |2     |Junior    |15000 |25000               |
// |3     |Junior    |40000 |65000               |
// |4     |Senior    |16000 |16000               |
// |5     |Senior    |20000 |36000               |
// |6     |Senior    |50000 |86000               |
// +------+----------+------+--------------------+


// cte2
// +------+----------+------+--------------------+------------+
// |emp_id|experience|salary|salary_by_experience|total_budget|
// +------+----------+------+--------------------+------------+
// |4     |Senior    |16000 |16000               |16000       |
// |5     |Senior    |20000 |36000               |36000       |
// |1     |Junior    |10000 |10000               |46000       |
// |2     |Junior    |15000 |25000               |61000       |
// |3     |Junior    |40000 |65000               |101000      |
// +------+----------+------+--------------------+------------+


// final result
// +------+----------+------+
// |emp_id|experience|salary|
// +------+----------+------+
// |1     |Junior    |10000 |
// |2     |Junior    |15000 |
// |4     |Senior    |16000 |
// |5     |Senior    |20000 |
// +------+----------+------+